In [ ]:
# Imports
import pandas as pd
import re

In [86]:
# Read the Excel file
county_raw = pd.read_excel('1997-2017年中国县级尺度碳排放.xlsx', sheet_name='Unit (Million tons)')
city_2020_raw = pd.read_excel('20221016_164053_474470.xlsx', sheet_name='二氧化碳排放(万吨)')
city_2015_raw = pd.read_excel('20221016_164004_290052.xlsx', sheet_name='二氧化碳排放(万吨)')
city_2010_raw = pd.read_excel('20221016_163454_333468.xlsx', sheet_name='二氧化碳排放(万吨)')
city_2005_raw = pd.read_excel('20221016_163805_882972.xlsx', sheet_name='二氧化碳排放(万吨)')

In [87]:
# Define a function to combine the province and city names
def combine_province_city(row):
    if row['省份\nProvince'] in ['北京', '天津', '重庆', '上海']:
        return row['省份\nProvince']
    else:
        if row['城市名称\nCity'] in ['湖北直辖县', '河南直辖县', '海南直辖县']:
            return row['城市名称\nCity']
        else:
            return row['省份\nProvince'] + row['城市名称\nCity']
    
# Define a function to remove non-Chinese characters
def remove_non_chinese(text):
    return re.sub('[^\u4e00-\u9fff]', '', text)

# Define a function to prepare the city dataset
def prepare_city(df):
    df['城市'] = df.apply(combine_province_city, axis=1)
    selected_columns = df[[
        '城市', '农业\nAgriculture', '服务业\nService', '工业\nIndustrial total',
        '城镇生活\nUrban household', '农村生活\nRural household', '生活\nHousehold',
        '交通\nTransport', '直接排放\nDirect'
    ]]
    selected_columns.columns = map(remove_non_chinese, selected_columns.columns)
    selected_columns['产业'] = selected_columns['农业'] + selected_columns['服务业'] + selected_columns['工业']
    final_columns = ['城市', '农业', '工业', '服务业', '产业', '城镇生活', '农村生活', '生活', '交通', '直接排放']
    final_df = selected_columns[final_columns]
    
    return final_df

# Select data
city_2020_select = prepare_city(city_2020_raw)
city_2015_select = prepare_city(city_2015_raw)
city_2010_select = prepare_city(city_2010_raw)
city_2005_select = prepare_city(city_2005_raw)

city_2020 = city_2020_select[['城市', '直接排放']].rename(columns={'直接排放': '2020'})
city_2015 = city_2015_select[['城市', '直接排放']].rename(columns={'直接排放': '2015'})
city_2010 = city_2010_select[['城市', '直接排放']].rename(columns={'直接排放': '2010'})
city_2005 = city_2005_select[['城市', '直接排放']].rename(columns={'直接排放': '2005'})

# 使用“城市”列作为键合并这四个数据集
from functools import reduce
dfs = [city_2005, city_2010, city_2015, city_2020]
city_allyear = reduce(lambda left, right: pd.merge(left, right, on='城市', how='outer'), dfs)

C:\Users\admin\AppData\Local\Temp\ipykernel_17932\1299085883.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['产业'] = selected_columns['农业'] + selected_columns['服务业'] + selected_columns['工业']
C:\Users\admin\AppData\Local\Temp\ipykernel_17932\1299085883.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['产业'] = selected_columns['农业'] + selected_columns['服务业'] + selected_columns['工业']
C:\Users\admin\AppData\Local\Temp\ipykernel_17932\1299085883.py:24: SettingWithCopyWar

In [88]:
# Define a function to simplify the administrative name
def simplify_administrative_name(name):
    ethnic_groups = [
        '汉族', '蒙古族', '回族', '藏族', '维吾尔族', '苗族', '彝族', '壮族',
        '布依族', '朝鲜族', '满族', '侗族', '瑶族', '白族', '土家族', '哈尼族',
        '哈萨克族', '傣族', '黎族', '傈僳族', '佤族', '畲族', '高山族', '拉祜族',
        '水族', '东乡族', '纳西族', '景颇族', '柯尔克孜族', '土族', '达斡尔族', '仫佬族',
        '羌族', '布朗族', '撒拉族', '毛难族', '仡佬族', '锡伯族', '阿昌族', '普米族',
        '塔吉克族', '怒族', '乌孜别克族', '俄罗斯族', '鄂温克族', '德昂族', '保安族', '裕固族',
        '京族', '塔塔尔族', '独龙族', '鄂伦春族', '赫哲族', '门巴族', '珞巴族', '基诺族', '维吾尔', '哈萨克'
    ]
    ethnic_pattern = '|'.join(ethnic_groups)
    if pd.isnull(name) or not isinstance(name, str):
        return None  
    name = re.sub(f'({ethnic_pattern}).*$', '', name)
    name = re.sub(r'巴音郭楞蒙古', '巴音郭楞', name)
    name = re.sub(r'博尔塔拉蒙古', '博尔塔拉', name)
    name = re.sub(r'(.*?)(自治区|自治州|自治县|市|省|地区|特别行政区|县|旗|自治旗|区|林区|特区|盟)?$', r'\1', name)
    return name.strip()


# 应用函数到 'City Name' 和 'Province Name' 列
county_raw['City Name Simplified'] = county_raw['City Name'].apply(simplify_administrative_name)
county_raw['Province Name Simplified'] = county_raw['Province Name'].apply(simplify_administrative_name)

# Define a function to combine the province and city names
def combine_province_city_county(row):
    if row['Province Name Simplified'] in ['北京', '天津', '重庆', '上海']:
        return row['Province Name Simplified']
    else:
        return f"{row['Province Name Simplified']}{row['City Name Simplified']}"

# Reshape the county dataset
county_raw['城市'] = county_raw.apply(combine_province_city_county, axis=1)
county_raw.rename(columns={'County Name': '县'}, inplace=True)
pattern = re.compile(r'.*?\d{4}.*?')
columns_with_four_digits = [col for col in county_raw.columns if pattern.match(col)]
for col in columns_with_four_digits:
    county_raw[col] = county_raw[col].astype(float) * 100
columns_to_keep = ['城市', '县'] + columns_with_four_digits
county_select = county_raw[columns_to_keep]
county_select.columns = [col[1:] if col.startswith('C') and any(char.isdigit() for char in col) else col for col in county_select.columns]
county_select = county_select.dropna()

In [89]:
# Group the county dataset
columns_to_aggregate = [col for col in county_select.columns if col != '县']
county_grouped = county_select[columns_to_aggregate].groupby('城市').sum()
county_grouped = county_grouped.reset_index()

# Compare two datasets
cities_in_county_grouped = set(county_grouped['城市'].unique())
cities_in_city = set(city_allyear['城市'].unique())
cities_only_in_county_grouped = cities_in_county_grouped - cities_in_city
cities_only_in_city = cities_in_city - cities_in_county_grouped
print("只在county_grouped中存在的城市:", cities_only_in_county_grouped)
print("只在city中存在的城市:", cities_only_in_city)

# Merge two datasets
county_grouped.columns = ['A' + col if col != '城市' else col for col in county_grouped.columns]
city_allyear.columns = ['B' + col if col != '城市' else col for col in city_allyear.columns]
city = pd.merge(county_grouped, city_allyear, on='城市', how='inner')

# Clean the County dataset
cities_in_city = set(city['城市'])
county = county_select[county_select['城市'].isin(cities_in_city)]
county.to_csv('analysis\county.csv', index=False)
city.to_csv('analysis\city.csv', index=False)

只在county_grouped中存在的城市: {'安徽巢湖', '海南五指山', '海南白沙', '海南东方', '海南澄迈', '新疆石河子', '新疆五家渠', '湖北潜江', '河南济源', '海南陵水', '海南临高', '湖北天门', '新疆阿拉尔', '海南昌江', '海南定安', '海南文昌', '山东莱芜', '湖北仙桃', '新疆图木舒克', '海南琼海', '海南乐东', '海南屯昌', '海南万宁', '湖北神农架'}
只在city中存在的城市: {'西藏阿里', '河南直辖县', '海南直辖县', '湖北直辖县', '西藏昌都', '西藏日喀则', '西藏林芝', '西藏拉萨', '西藏那曲', '西藏山南'}


In [91]:
# Calculate the proportion of county to city
year_columns_county = [col for col in county.columns if col.isdigit()]
county_proportion = county[['城市', '县']].copy()

for year in year_columns_county:
    year_column_city = f'A{year}'
    merged_df = county[['城市', '县', year]].merge(city[['城市', year_column_city]], on='城市')
    proportion_column_name = f'Proportion_{year}'
    merged_df[proportion_column_name] = merged_df[year] / merged_df[year_column_city]
    county_proportion = county_proportion.merge(merged_df[['城市', '县', proportion_column_name]], on=['城市', '县'], how='left')

county_proportion.columns = [col.replace('Proportion_', '') for col in county_proportion.columns]